# Install Modules

In [ ]:
# !{sys.executable} -m pip install selenium

# Load Modules

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import sys
import pandas as pd
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import glob
import time
from selenium.webdriver.chrome.options import Options

# Track Progress

In [1102]:
'''

State Track

Alabama - AL: shown: 78370, get: 78368
Alaska - AK: shown: 30247, get: 30247
Arizona - AZ: shown: 43853, get: 43850
Arkansas - AR: shown: 60605, get: 60601
California - CA: shown: 73952, get: 73950
Colorado - CO: shown: 77877, get: 77875
Connecticut - CT: shown: 95036, get: 95034
Delaware - DE: shown: 56979, get: 56979
Florida - FL: Not Participating
Georgia - GA: shown: 111089, get: 111045
Hawaii - HI: shown: 16604, get: 16604
Idaho - ID: shown: 68703, get: 68692 
Illinois - IL: shown: 95548, get: 95542
Indiana - IN: shown: 84548, get: 84536
Iowa - IA: shown: 84548, get 84536
Kansas - KS: shown: 67508, get: 67508
Kentucky - KY: shown: 23865, get: 23865
Louisiana - LA: shown: 42752, get: 42752
Maine - ME: 62707, 62676
Maryland - MD: 111593, 111478
Massachusetts - MA: Not Participating
Michigan - MI: 40563, 40561
Minnesota - MN: 12018, 12018
Mississippi - MS: 61785, 61776
Missouri - MO: 103455, 103448
Montana - MT: 73646, 73638
Nebraska - NE: 70104, 70103
Nevada - NV: 67416, 67413
New Hampshire - NH: 68834, 68834
New Jersey - NJ: shown: 46766, get: 46765
New Mexico - NM: 60263, 60259
New York - NY: 56200, 56200
North Carolina - NC: 90834, 90834
North Dakota - ND: 59931, 59931
Ohio - OH: 70882, 70882
Oklahoma - OK: 110004, 109999
Oregon - OR: 79944, 79942
Pennsylvania - PA: 100629, 100622
Rhode Island - RI: 42177, 42174
South Carolina - SC: 63910, 63910
South Dakota - SD: 62300, 62290
Tennessee - TN: 79009, 79002
Texas - TX: 73149, 73149
Utah - UT: 80767, 80763
Vermont - VT: 48573, 48572
Virginia - VA: 73253, 73251
Washington - WA: 59447, 59447
West Virginia - WV: 54866, 54843
Wisconsin - WI: 115712, 115707
Wyoming - WY: 37015, 37010


'''

'\n\nState Track\n\nAlabama - AL: shown: 78370, get: 78368\nAlaska - AK: shown: 30247, get: 30247\nArizona - AZ: shown: 43853, get: 43850\nArkansas - AR: shown: 60605, get: 60601\nCalifornia - CA: shown: 73952, get: 73950\nColorado - CO: shown: 77877, get: 77875\nConnecticut - CT: shown: 95036, get: 95034\nDelaware - DE: shown: 56979, get: 56979\nFlorida - FL: Not Participating\nGeorgia - GA: shown: 111089, get: 111045\nHawaii - HI: shown: 16604, get: 16604\nIdaho - ID: shown: 68703, get: 68692 \nIllinois - IL: shown: 95548, get: 95542\nIndiana - IN: shown: 84548, get: 84536\nIowa - IA: shown: 84548, get 84536\nKansas - KS: shown: 67508, get: 67508\nKentucky - KY: shown: 23865, get: 23865\nLouisiana - LA: shown: 42752, get: 42752\nMaine - ME: 62707, 62676\nMaryland - MD: 111593, 111478\nMassachusetts - MA: Not Participating\nMichigan - MI: 40563, 40561\nMinnesota - MN: 12018, 12018\nMississippi - MS: 61785, 61776\nMissouri - MO: 103455, 103448\nMontana - MT: 73646, 73638\nNebraska - NE

# Web Crawling

In [28]:
s = Service('/Users/Documents/chromedriver-mac-arm64/chromedriver')

In [29]:
state = 'WY'

In [30]:
url = 'https://filingaccess.serff.com/sfa/home/'+state


In [31]:
driver = webdriver.Chrome(service=s)

# Open a webpage
driver.get(url)

In [1399]:
# Browse to the current page

In [1400]:
def get_index_page_v2(page_number, state):
    html_content = driver.page_source
    # Parse the HTML
    soup = BeautifulSoup(html_content, 'lxml')
    paginator_span = soup.find('span', class_='ui-paginator-current')

    # Extract the text inside the span
    page_info = paginator_span.text.strip()  # This will give you '(1 of 48)'

    # To extract just the numbers and format it, you can do some additional processing:
    import re

    # Using regular expression to extract numbers
    numbers = re.findall(r'\d+', page_info)
    if numbers:
        current_page, total_pages = numbers
        current_page = int(current_page)
        total_pages = int(total_pages)
        print(f"Current Page: {current_page}, Total Pages: {total_pages}")
    else:
        print("No page information found.")
    # Find the table
    table = soup.find('table', attrs={'role': 'grid'})

    # Extract and print each row of the table
    column_texts_list = []
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        # Extract text from each column
        column_texts = [col.text.strip() for col in columns]
        column_texts_list.append(column_texts)

    column_texts_list = [x for x in column_texts_list if len(x)!=0]

    result = pd.DataFrame()
    
    result = pd.DataFrame({'Company Name':[x[0] for x in column_texts_list],
                      'NAIC Company Code':[x[1] for x in column_texts_list],
                      'Insurance Product Name':[x[2] for x in column_texts_list],
                      'Sub Type of Insurance':[x[3] for x in column_texts_list],
                      'Filing Type':[x[4] for x in column_texts_list],
                      'Filing Status':[x[5] for x in column_texts_list],
                      'SERFF Tracking Number':[x[6] for x in column_texts_list]})
        # Check if the folder exists
    folder_path = '/Users/qiaojiang/Documents/Research/SERFF/'+state
    if not os.path.exists(folder_path):
        # Create the folder if it does not exist
        os.makedirs(folder_path)
        print("Folder created:", folder_path)
    
    result.to_csv(folder_path+'/index_page_'+str(current_page)+'.csv', index = False)
    print('Index file saved!')

In [1401]:
def get_all_index_page_v2(state):
    html_content = driver.page_source
    # Parse the HTML
    soup = BeautifulSoup(html_content, 'lxml')
    paginator_span = soup.find('span', class_='ui-paginator-current')

    # Extract the text inside the span
    page_info = paginator_span.text.strip()  # This will give you '(1 of 48)'

    # To extract just the numbers and format it, you can do some additional processing:
    import re

    # Using regular expression to extract numbers
    numbers = re.findall(r'\d+', page_info)
    if numbers:
        current_page, total_pages = numbers
        current_page = int(current_page)
        total_pages = int(total_pages)
        print(f"Current Page: {current_page}, Total Pages: {total_pages}")
    else:
        print("No page information found.")
    
    old_html_content = html_content
    
    for i in range(total_pages):
        page_number = i+1
        print(f'This is supposed to be page {page_number}')
        get_index_page_v2(page_number, state)
        button = WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="j_idt25:filingTable_paginator_top"]/a[3]')))
        # Click the button
        button.click()
        time.sleep(3.5)
        
        new_html_content = driver.page_source
        
        if old_html_content==new_html_content:
            print('Wait for the new page')
            time.sleep(3.5)

In [ ]:
get_all_index_page_v2(state)

In [1403]:
state

'WY'

In [1405]:
folder_path = '/Users/Documents/Research/SERFF/'+state


In [1406]:
index_files = [x for x in glob.glob(folder_path+'/*.csv') if 'master' not in x]

In [ ]:
result_list = []
for file in index_files:
    result_list.append(pd.read_csv(file))
    print(pd.read_csv(file).shape)

In [1408]:
result_df = pd.concat(result_list)

In [1409]:
result_df.shape

(37015, 7)

In [1410]:
result_df.drop_duplicates(inplace = True)


In [1411]:
result_df.shape # 46765


(37010, 7)

In [1412]:
result_df.to_csv('/Users/Documents/Research/SERFF/'+state+'/master_index_'+state+'.csv', index = False)

# Organize the Master Index Files for All States

In [16]:
all_master_index_files = glob.glob('/Users/Documents/Research/SERFF/**/master_index*.csv', recursive = True)

In [17]:
len(all_master_index_files)

48

In [19]:
all_master_index_list = []
for file in all_master_index_files:
    temp = pd.read_csv(file)
    temp['State'] = file.split('/')[-2]
    all_master_index_list.append(temp)
all_master_index_df = pd.concat(all_master_index_list)

In [20]:
all_master_index_df.shape

(3245779, 8)

In [21]:
all_master_index_df.head()

,Company Name,NAIC Company Code,Insurance Product Name,Sub Type of Insurance,Filing Type,Filing Status,SERFF Tracking Number,State
0,XL Specialty Insurance Company,37885.0,Excess Workers Compensation,16.0003 Excess WC,Form,Closed - Approved,XLAM-129754727,VT
1,XL Specialty Insurance Company,37885.0,Excess Workers Compensation,16.0003 Excess WC,Form,Closed - Approved,XLAM-132249557,VT
2,XL Specialty Insurance Company,37885.0,Excess Workers Compensation,16.0003 Excess WC,Form,Closed - Approved,XLAM-129498898,VT
3,XL Specialty Insurance Company,37885.0,Fetch Pet Health Insurance Program 2023 Revisi...,09.0004 Pet Insurance Plans,Form/Rate/Rule,Closed - Approved,XLAM-133698612,VT
4,XL Specialty Insurance Company,37885.0,Fetch Pet Health Insurance Program 2023 Revisi...,09.0004 Pet Insurance Plans,Rate/Rule,Closed - Rate/Rule-P&C Use and File,XLAM-133863357,VT


In [22]:
all_master_index_df.to_csv('/Users/Documents/Research/SERFF/master_index_all_states.csv', index = False)